# 2D Heat Equation on a Square with h- and p-Refinement Study

This notebook solves the 2D heat equation on a square domain \([0, 1] \times [0, 1]\) and compares the maximum temperature at \( t = 0.5 \) for different mesh resolutions (h-refinement) and element types (p-refinement). The problem is:
\begin{align}
\frac{\partial u}{\partial t} &= \kappa \Delta u, \quad (x, y) \in [0, 1] \times [0, 1], \quad t > 0 \\
u(x, y, 0) &= \sin(\pi x) \sin(\pi y), \\
u &= 0 \quad \text{on } \partial \Omega,
\end{align}
with \( \kappa = 1 \). The analytical solution is:
\[ u(x, y, t) = e^{-2 \pi^2 t} \sin(\pi x) \sin(\pi y) \]
The maximum temperature \( u_{\text{max}}(t) = e^{-2 \pi^2 t} \) (at \( (x, y) = (0.5, 0.5) \)) is compared between numerical and analytical solutions.

**h-Refinement**: Mesh resolutions \( nx \times ny = 20\times20, 40\times40, 60\times60, 100\times100 \).
**p-Refinement**: Element types:
- `nn3_tri`: Linear triangle (P1, 3 nodes).
- `nn6_tri`: Quadratic triangle (P2, 6 nodes).
- `nn4_quad`: Linear quadrilateral (Q1, 4 nodes).
- `nn8_quad`: Quadratic quadrilateral (Q2, 8 nodes).

The maximum temperature at \( t = 0.5 \) is plotted in a bar chart for comparison.

In [1]:
import ufl
import numpy as np
from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc
import matplotlib.pyplot as plt

# Define temporal parameters
T = 0.5
num_steps = 50
dt = T / num_steps
kappa = 1.0  # Diffusion coefficient

# Define mesh resolutions for h-refinement
mesh_sizes = [(20, 20), (40, 40), (60, 60), (100, 100)]

# Define element types for p-refinement
element_types = [
    ('triangle', 'P', 1),  # nn3_tri: Linear triangle (P1)
    ('triangle', 'P', 2),  # nn6_tri: Quadratic triangle (P2)
    ('quadrilateral', 'Q', 1),  # nn4_quad: Linear quadrilateral (Q1)
    ('quadrilateral', 'Q', 2)   # nn8_quad: Quadratic quadrilateral (Q2)
]
element_names = ['nn3_tri', 'nn6_tri', 'nn4_quad', 'nn8_quad']

The domain is meshed with either triangular or quadrilateral elements, varying resolution and polynomial degree.

In [2]:
# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x[0]) * np.sin(np.pi * x[1])

# Analytical maximum temperature
def analytical_max_temperature(t, kappa=1.0):
    return np.exp(-2 * kappa * np.pi**2 * t)

## Variational Problem and Solver Setup
Define the weak form and solver for the heat equation.

In [3]:
# Store results
results = {name: [] for name in element_names}
analytical_u_max = analytical_max_temperature(T, kappa)

# Loop over mesh sizes and element types
for nx, ny in mesh_sizes:
    for (cell_type, family, degree), elem_name in zip(element_types, element_names):
        # Create mesh
        cell_type_map = {'triangle': mesh.CellType.triangle, 'quadrilateral': mesh.CellType.quadrilateral}
        domain = mesh.create_rectangle(
            MPI.COMM_WORLD, [np.array([0, 0]), np.array([1, 1])], [nx, ny], cell_type_map[cell_type]
        )
        
        # Create function space
        V = fem.functionspace(domain, (family, degree))
        
        # Create initial condition
        u_n = fem.Function(V)
        u_n.interpolate(initial_condition)
        
        # Create boundary condition
        fdim = domain.topology.dim - 1
        boundary_facets = mesh.locate_entities_boundary(
            domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
        bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

        # Define variational problem
        u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
        f = fem.Constant(domain, PETSc.ScalarType(0))
        a = u * v * ufl.dx + kappa * dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
        L = (u_n + dt * f) * v * ufl.dx
        bilinear_form = fem.form(a)
        linear_form = fem.form(L)

        # Assemble matrix and vector
        A = assemble_matrix(bilinear_form, bcs=[bc])
        A.assemble()
        b = create_vector(linear_form)

        # Create solver
        solver = PETSc.KSP().create(domain.comm)
        solver.setOperators(A)
        solver.setType(PETSc.KSP.Type.PREONLY)
        solver.getPC().setType(PETSc.PC.Type.LU)

        # Define solution variable
        uh = fem.Function(V)
        uh.interpolate(initial_condition)

        # Time-stepping
        t = 0
        for i in range(num_steps):
            t += dt

            # Update right-hand side
            with b.localForm() as loc_b:
                loc_b.set(0)
            assemble_vector(b, linear_form)
            apply_lifting(b, [bilinear_form], [[bc]])
            b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
            set_bc(b, [bc])

            # Solve
            solver.solve(b, uh.x.petsc_vec)
            uh.x.scatter_forward()

            # Update previous solution
            u_n.x.array[:] = uh.x.array

        # Compute maximum temperature
        u_max_num = np.max(uh.x.array)
        results[elem_name].append(u_max_num)

        # Clean up
        solver.destroy()
        b.destroy()
        A.destroy()

## Maximum Temperature Comparison
Plot the maximum temperature at \( t = 0.5 \) for each mesh resolution and element type, compared to the analytical solution.

In [4]:
# Plotting
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.2
mesh_labels = [f'{nx}x{ny}' for nx, ny in mesh_sizes]
x = np.arange(len(mesh_sizes))

# Plot bars for each element type
for i, elem_name in enumerate(element_names):
    ax.bar(x + i * bar_width, results[elem_name], bar_width, label=elem_name)

# Plot analytical solution as a horizontal line
ax.axhline(y=analytical_u_max, color='r', linestyle='--', label='Analytical')

# Customize plot
ax.set_xlabel('Mesh Resolution (nx × ny)')
ax.set_ylabel('Maximum Temperature at t = 0.5')
ax.set_title('Maximum Temperature Comparison: h- and p-Refinement')
ax.set_xticks(x + bar_width * (len(element_names) - 1) / 2)
ax.set_xticklabels(mesh_labels)
ax.legend()
ax.grid(True, which='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Print relative errors
for elem_name in element_names:
    for i, (nx, ny) in enumerate(mesh_sizes):
        u_max_num = results[elem_name][i]
        relative_error = abs(u_max_num - analytical_u_max) / analytical_u_max
        print(f'Element: {elem_name}, Mesh: {nx}x{ny}, Relative Error: {relative_error:.2e}')